In [2]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

from coeditor.dataset import TokenizedEditDataset, is_repetitive_edit
from coeditor.model import *
from coeditor.encoding import TokenizedEdit, decode_tokens, WindowArgs, tokens_to_change
import shutil
import random
from prepare_data import make_or_load_datasets

os.chdir(proj_root())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
wargs = WindowArgs(4096)

# test_data_name = "medium"
test_data_name = "SPOT"
datasets = make_or_load_datasets(test_data_name)
test_data = datasets["test"]
test_data = test_data.map(lambda e: e.truncate_ctx(wargs))

rep_edits = pfilter(is_repetitive_edit, test_data.all_edits())
print("Total edits: ", len(list(test_data.all_edits())))
print("Repetitive edits: ", len(rep_edits))


100%|██████████| 4/4 [03:15<00:00, 48.89s/chunk] 


Tokenized dataset saved to: /mnt/nas/jiayi/coeditor/datasets/SPOT/AnalysisBasedEditEncoder(window=(max_window_size=4096), extra_ctx_size=1000, extra_ctx_names=('usees', 'post-usees'))
Dataset stats:
==================== test ====================
n_projects: 1
n_edits: 1343
input_size:
   mean: 2762.8
   median: 2769
   min: 70
   max: 4096
output_size:
   mean: 110.08
   median: 72
   min: 6
   max: 1705


100%|██████████| 1343/1343 [00:00<00:00, 1491.52it/s]

Total edits:  1343
Repetitive edits:  87


In [4]:
# model_dir = get_model_dir(trained=True) / "coeditor-medium-collapse-shuffle"
model_dir = get_model_dir(trained=True) / "coeditor-medium-analysis-post_usees"
model = CoeditorModel.load_pretrained(model_dir)
model.to("cuda:0")

eval_args = EvalArgs(4096 * 2, WindowArgs(4096))


In [5]:
rep_data = TokenizedEditDataset.from_edits(rep_edits)
rep_result = model.eval_loss_on_data(rep_data, eval_args)
display(rep_result)


evaluate loss: 100%|██████████| 36/36 [00:20<00:00,  1.76batch/s]


{'loss_per_ex': (mean=2.0822, weight=87),
 'loss_per_tk': (mean=0.032216, weight=5623),
 'prob_per_ex': (mean=0.67076, weight=87)}

In [7]:
sub_data = TokenizedEditDataset.from_edits(random.sample(test_data.all_edits(), 50))
dec_args = DecodingArgs(num_beams=1)
dec_result = model.predict_on_data(sub_data, eval_args, dec_args)
display(dec_result.exact_match_accuracy())


decoding: 100%|██████████| 19/19 [00:25<00:00,  1.36s/batch]


(mean=0.14, weight=50)

In [8]:
max_saved_samples = 200
random.seed(42)
exs_to_save = list(range(len(dec_result.predictions)))
random.shuffle(exs_to_save)
out_dir = Path("output/inspect_coeditor") / model_dir.name
dec_result.save_examples_to_dir(out_dir, exs_to_save[:max_saved_samples])

saving examples: 100%|██████████| 50/50 [00:00<00:00, 81.92it/s]
